In [3]:
import urllib2, json
import pandas as pd
import numpy as np
import datetime, time
from SECRET_KEY import secret_key

In [18]:
# Load TAZ records with x and y coordinates attached
taz = pd.read_csv(r'R:\Brice\googlemaps\taz_xy\taz_xy.txt')

# Create a Google-formatted coordinates field
taz['g_coord'] = taz['y_gps'].astype('str') + ',' + taz['x_gps'].astype('str')



# set standard departure time for tomorrow at 8 AM
dep_hr = 8
dep_time = datetime.datetime.now()
dep_time = dep_time.replace(hour=dep_hr,day=dep_time.day+1)
dep_time = str(int(time.mktime(dep_time.timetuple())))

# Skims can be auto, transit, bike, or walk
mode = 'auto'

# Create empty skims to be filled with results
skims = {'auto_8_dist': np.zeros([4000,4000]),
    'auto_8_time_ff': np.zeros([4000,4000]),
    'auto_8_time_congested': np.zeros([4000,4000])}

# list of TAZ IDs to find data for, max size of 25 per request
# taz_list = [range(1,25)]
# taz_list = [xrange(i,i+25) for i in range(1,4000,25)]
# taz_lists = [xrange(i,i+25) for i in range(1,200,25)]
taz_lists = [xrange(i,i+25) for i in range(50,200,25)]

In [26]:
otaz = taz_lists[0][0]
# print otaz
    
results = {}
urlfeed = ""
print otaz
origin = taz[taz['ID'] == otaz]['g_coord'].values[0]
# get list of different destinations
destination = ''
dtaz_list = []
for dtaz in taz_list:
    if otaz != dtaz:    # skip intrazonal trips where otaz==dtaz
        destination += taz[taz['ID'] == dtaz]['g_coord'].values[0] + '|'
        dtaz_list.append(dtaz)
    # remove trailing |
destination = destination[:-1]

urlfeed += "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin+"&destinations="+destination+ \
        "&mode="+mode+"&departure_time="+dep_time+"&key="+secret_key+"&units=imperial"

# Fetch url and store
results[otaz] = json.loads(urllib2.urlopen(urlfeed).read())
results[otaz]['dtaz_list'] = taz_list

50


In [27]:
results

{50: {u'destination_addresses': [],
  'dtaz_list': xrange(300, 303),
  u'error_message': u'You have exceeded your daily request quota for this API.',
  u'origin_addresses': [],
  u'rows': [],
  u'status': u'OVER_QUERY_LIMIT'}}

In [7]:
for taz_list in taz_lists:

    # Look up 
    for otaz in taz_list:
        results = {}
        urlfeed = ""
        print otaz
        origin = taz[taz['ID'] == otaz]['g_coord'].values[0]
        # get list of different destinations
        destination = ''
        dtaz_list = []
        for dtaz in taz_list:
            if otaz != dtaz:    # skip intrazonal trips where otaz==dtaz
                destination += taz[taz['ID'] == dtaz]['g_coord'].values[0] + '|'
                dtaz_list.append(dtaz)
            # remove trailing |
        destination = destination[:-1]

        urlfeed += "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin+"&destinations="+destination+ \
                "&mode="+mode+"&departure_time="+dep_time+"&key="+secret_key+"&units=imperial"

        # Fetch url and store
        results[otaz] = json.loads(urllib2.urlopen(urlfeed).read())
        results[otaz]['dtaz_list'] = taz_list


        # loop through each origin
        for otaz, data in results.iteritems():
            # loop through each destination
            try:
                for i in xrange(len(results[otaz]['rows'][0]['elements'])):
                    dtaz = dtaz_list[i]
                    dist = results[otaz]['rows'][0]['elements'][i]['distance']['value']
                    time_ff = results[otaz]['rows'][0]['elements'][i]['duration']['value']    # free flow skim
                    time_cong = results[otaz]['rows'][0]['elements'][i]['duration_in_traffic']['value']    # congested

                    skims['auto_8_dist'][otaz-1][dtaz-1] = dist*0.000621371    # convert meters to miles
                    skims['auto_8_time_ff'][otaz-1][dtaz-1] = time_ff/60    # convert seconds to minutes
                    skims['auto_8_time_congested'][otaz-1][dtaz-1] = time_cong/60    # convert seconds to minutes
            except:
                print 'no values returned'

300
301
302


In [13]:
taz_lists = [xrange(i,i+25) for i in range(1,100,25)]

In [14]:
taz_lists

[xrange(1, 26), xrange(26, 51), xrange(51, 76), xrange(76, 101)]

In [17]:
taz_lists[1][-1]

50

In [28]:
#### Where are the largest trips occuring?

import h5py

In [35]:
daysim = h5py.File(r'R:\SoundCast\releases\TransportationFutures2010\outputs\daysim_outputs.h5')

In [38]:
trip = pd.DataFrame()
for field in daysim['Trip'].keys():
    trip[field] = daysim['Trip'][field][:]

In [44]:
df = pd.DataFrame(trip.groupby(['otaz','dtaz']).count()['day'])

day
otaz dtaz     
1    1      67
     2      95
     3      54
     4      82
     5      25
     6      16
     7      23
     8      23
     9      10
     10      3
     11     13
     12      6
     13     14
     14     10
     15      9
     16     10
     17      8
     18      3
     19      5
     20     15
     21      5
     22      5
     23      7
     24     12
     25     13
     26     23
     27     21
     28      8
     29     32
     30     12
...        ...
3906 2573    1
     2579    1
     2587    1
     2589    1
     2590    1
     2593    2
     2600    1
     2605    2
     2609    1
     2612    2
     2617    1
     2619    1
     2622    1
     2624    1
     2625    1
     2626    1
     2629    1
     2642    2
     2643    2
     2649    1
     2651    1
     2653    2
     2657    1
     2659    1
     2662    4
     2663    5
     2664    1
     2665    3
     2666    1
     2667    2

[2591904 rows x 1 columns]

In [53]:
len(trip)

15397776